In [ ]:
import sys
sys.path.append('../')
from SQLConnection import LocalConnection
from SQLConnection import AWSConnection
sys.path.append('../databases')
import mysql_aws_credentials as aws
from Format import Format
import re

In [ ]:
cons_test_users = aws_conn.get_users_by_pol_label('conservative')
libs_test_users = aws_conn.get_users_by_pol_label('liberal')
cons_train_users = local_conn.get_users_by_pol_label('conservative')
libs_train_users = local_conn.get_users_by_pol_label('liberal')

In [ ]:
print(cons_train_users.__len__())
print(libs_train_users.__len__())
cons_train_users[12]

In [ ]:
test_tweets = local_conn.get_all_tweets_by('sentedcruz')
test_tweets1 = local_conn.get_all_tweets_by('aoc')

In [ ]:
test_tweet = test_tweets[0][1]

In [ ]:
import nltk
nltk.download('stopwords')
import string

In [ ]:
# import nltk
# nltk.download('stopwords')
# import string
# stopwords = nltk.corpus.stopwords.words('english')
# def remove_stopwords(input_text):
#     stopwords = set(nltk.corpus.stopwords.words('english'))
#     stopwords.add('it\'s')
#     stopwords.add('w/')
#     stopwords.add('\'s')
#     input_text_split = input_text.split()
#     output_text_split = [word for word in input_text_split if word not in stopwords]
#     return " ".join(output_text_split)

In [ ]:
denoised = []
denoised1 = []
for tweet in test_tweets:
    denoised.append(Format.denoise_tweet(tweet[1]))
    
for tweet in test_tweets1:
    denoised1.append(Format.denoise_tweet(tweet[1]))

In [ ]:
tokens = []
for i,d in enumerate(denoised):
    tokens.extend(d.split())
    
tokens1 = []
for i,d in enumerate(denoised1):
    print(i, test_tweets1[i][1])
    print('////')
    print(d,'\n')
    tokens1.extend(d.split())

In [ ]:
import collections
count = collections.Counter(tokens)
# count.most_common(50)

In [ ]:
import collections
count1 = collections.Counter(tokens1)
# count1.most_common(50)

In [ ]:
print(t, '\n')
t = Format.remove_hashtags(t)
print(t, '\n')
t = Format.remove_mentions(t)
print(t, '\n')
t = remove_stopwords(t)
print(t, '\n')
t = Format.remove_picture_links(t)
print(t, '\n')
t = t.translate(str.maketrans('','',string.punctuation + '—' + '“' + '…'))
print(t, '\n')
t.split()

In [ ]:
tweet_text = test_tweets[73][1]
tweet_text

In [ ]:
tweet_text = Format.remove_picture_links(tweet_text)
print(tweet_text, '\n')
tweet_text = tweet_text.lower()
print(tweet_text, '\n')
tweet_text = Format.remove_hashtags(tweet_text)
print(tweet_text, '\n')
tweet_text = Format.remove_mentions(tweet_text)
print(tweet_text, '\n')


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.metrics import classification_report
from sklearn import svm 
from sklearn.model_selection import train_test_split

In [ ]:
cons_test_users = aws_conn.get_users_by_pol_label('conservative')
libs_test_users = aws_conn.get_users_by_pol_label('liberal')
cons_train_users = local_conn.get_users_by_pol_label('conservative')
libs_train_users = local_conn.get_users_by_pol_label('liberal')

In [ ]:
print(len(cons_test_users))
print(len(libs_test_users))
print(len(cons_train_users))
print(len(libs_train_users))

In [ ]:
# ('sentedcruz', 1074480192, '1074480192', 'conservative', None)
corpus = []
for i, user in enumerate(cons_train_users):
    tweets = local_conn.get_all_tweets_by(user[0])
    tweets = [x[1] for x in tweets]
    tweets = " ".join(tweets)
    tweets = denoise_tweet(tweets)
    corpus.append((user[0],tweets, user[3]))
    
for user in libs_train_users:
    tweets = local_conn.get_all_tweets_by(user[0])
    tweets = [x[1] for x in tweets]
    tweets = " ".join(tweets)
    tweets = denoise_tweet(tweets)
    corpus.append((user[0], tweets, user[3]))

In [ ]:
for i, user in enumerate(cons_test_users):
    tweets = aws_conn.get_all_tweets_by(user[0])
    tweets = [x[1] for x in tweets]
    tweets = " ".join(tweets)
    tweets = denoise_tweet(tweets)
    corpus.append((user[0], tweets, user[3]))
    
for i, user in enumerate(libs_test_users):
    tweets = aws_conn.get_all_tweets_by(user[0])
    tweets = [x[1] for x in tweets]
    tweets = " ".join(tweets)
    tweets = denoise_tweet(tweets)
    corpus.append((user[0], tweets, user[3]))

In [ ]:
len(corpus)


In [ ]:
data = []
labels = []
for row in corpus:
    data.append(row[1])
    labels.append((row[0], row[2]))

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size = 0.1, random_state=6)

In [ ]:
y_train_username = []
y_train_label = []

for y in y_train:
    y_train_username.append(y[0])
    y_train_label.append(y[1])

In [ ]:
print(len(y_train_username))
print(len(y_train_label))

In [ ]:
print(len(y_test_username))
print(len(y_test_label))

In [ ]:
# data = test_data + train_data
# labels = test_labels + train_labels
# x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size = 0.33, random_state=42)

In [ ]:
# Create feature vectors 
vectorizer = TfidfVectorizer(min_df=4, max_df=0.9)
# Train the feature vectors
train_vectors = vectorizer.fit_transform(x_train) #(train_data)
# Apply model on test data 
test_vectors = vectorizer.transform(x_test)#(test_data)

In [ ]:
# Perform classification with SVM, kernel=linear 
model = svm.SVC(kernel='linear') 
model.fit(train_vectors, y_train_label) 
prediction = model.predict(test_vectors)


In [ ]:
print(len(prediction))

In [ ]:
for i, pred in enumerate(prediction):
    if y_test_label[i] != pred:
        print(y_test_username[i], pred)

In [ ]:
print (classification_report(y_test_label, prediction))
# print(len(y_test_label))

# START


In [4]:
import sys
sys.path.append('../')
from SQLConnection import LocalConnection
from SQLConnection import AWSConnection
sys.path.append('../databases')
import mysql_aws_credentials as aws
from Format import Format
import re
import nltk
nltk.download('stopwords')
import string
import joblib

from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.metrics import classification_report
from sklearn import svm 
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/knownastron/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
local_conn = LocalConnection('../databases/main.db')
aws_conn = AWSConnection(aws.HOST, aws.PORT, aws.DATABASE_NAME, aws.USER, aws.PASSWORD)

cons_test_users = aws_conn.get_users_by_pol_label('conservative')
libs_test_users = aws_conn.get_users_by_pol_label('liberal')
cons_train_users = local_conn.get_users_by_pol_label('conservative')
libs_train_users = local_conn.get_users_by_pol_label('liberal')

In [10]:
corpus = []
#add all tweet data to corpu
for i, user in enumerate(cons_train_users):
    tweets = local_conn.get_all_tweets_by(user[0])
    tweets = [x[1] for x in tweets] #extract just the tweet text
    tweets = " ".join(tweets) #join all tweets into one big blob
    tweets = Format.denoise_tweet(tweets)
    corpus.append((user[0], tweets, user[3])) #usernames, tweet blob, label
    
for user in libs_train_users:
    tweets = local_conn.get_all_tweets_by(user[0])
    tweets = [x[1] for x in tweets]
    tweets = " ".join(tweets)
    tweets = Format.denoise_tweet(tweets)
    corpus.append((user[0], tweets, user[3]))

In [11]:
for i, user in enumerate(cons_test_users):
    tweets = aws_conn.get_all_tweets_by(user[0])
    tweets = [x[1] for x in tweets]
    tweets = " ".join(tweets) #join all tweets into one big blob
    tweets = Format.denoise_tweet(tweets)
    corpus.append((user[0], tweets, user[3]))
    
for i, user in enumerate(libs_test_users):
    tweets = aws_conn.get_all_tweets_by(user[0])
    tweets = [x[1] for x in tweets]
    tweets = " ".join(tweets)
    tweets = Format.denoise_tweet(tweets)
    corpus.append((user[0], tweets, user[3]))

In [12]:
data = []
labels = []
for row in corpus:
    data.append(row[1])
    labels.append((row[0], row[2]))

In [13]:
print(len(data))

629


In [87]:
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size = 0.10, random_state=8)

In [88]:
#create lists of usernames and labels for training and test data
y_train_username = []
y_train_label = []

for y in y_train:
    y_train_username.append(y[0])
    y_train_label.append(y[1])
    
y_test_username = []
y_test_label = []

for y in y_test:
    y_test_username.append(y[0])
    y_test_label.append(y[1])

In [89]:
# Create feature vectors 
vectorizer = TfidfVectorizer(min_df=4, max_df=0.9)
# Train the feature vectors
train_vectors = vectorizer.fit_transform(x_train) #(train_data)
# Apply model on test data 
test_vectors = vectorizer.transform(x_test)#(test_data)

In [90]:
# Perform classification with SVM, kernel=linear 
model = svm.SVC(kernel='linear') 
model.fit(train_vectors, y_train_label) 
prediction = model.predict(test_vectors)

In [91]:
print (classification_report(y_test_label, prediction))

              precision    recall  f1-score   support

conservative       0.97      1.00      0.98        32
     liberal       1.00      0.97      0.98        31

    accuracy                           0.98        63
   macro avg       0.98      0.98      0.98        63
weighted avg       0.98      0.98      0.98        63



In [86]:
for i, pred in enumerate(prediction):
    if y_test_label[i] != pred:
        print(y_test_username[i], pred)

emmaogreen conservative
mariodb liberal
yashar conservative
maggienyt conservative
senatorcollins liberal
buzzfeedben conservative


In [20]:
# joblib.dump(model, 'NLP_political_classifier.pkl')
# joblib.dump(vectorizer, 'tfid_vectorizer.pkl')
# NB_spam_model = open('NB_spam_model.pkl','rb')
# clf = joblib.load(NB_spam_model)

In [21]:
nlp_model = open('NLP_political_classifier.pkl','rb')
clf = joblib.load(nlp_model)

vectorizer_object = open('tfid_vectorizer.pkl', 'rb')
vectorizer = joblib.load(vectorizer_object)



In [69]:
con_test_str = 'america is the land of the free home of the brave god bless'
lib_test_str = 'lgbt rights union living wage'#'union living wage racism white people lgbt'
lib_test_list = [Format.denoise_tweet(lib_test_str)]
con_test_list = [Format.denoise_tweet(con_test_str)]

In [94]:
tweets = aws_conn.get_all_tweets_by('cenkuygur')
tweets = [x[1] for x in tweets]
tweets = " ".join(tweets)
tweets = Format.denoise_tweet(tweets)

In [95]:
tweets

'im tired bragging obama picked him essentially coin toss would appeal white voters more could easily evan bayh case anyone think biden would stage let alone leading field   right accuse negative records criminal justice reform played politics peoples lives back then redemption helping pass criminal justice reform scientists said bomb middle earth going go soon would question whether something imagine republican politicians said shouldnt anything donors bomb industry wish pipehitting outsider debate called central problem politics corporate donors buying politicians leading problems discussing  admitted made mistake believing bush iraq essential problem throughout career always believes republicans surprised every time dont right thing minimum thats dangerous naïveté lot empty fireworks debate think everyone essentially lost pick winner id say  im amused collective outrage media anyone would challenge s record especially left dont challenge facts like record deportations cant believe a

In [ ]:
lib_test_list = [tweets]

In [70]:
lib_test_vecs = vectorizer.transform(lib_test_list)
con_test_vecs = vectorizer.transform(con_test_list)

In [71]:
 = clf.predict(lib_test_vecs)

In [72]:
ya

array(['liberal'], dtype='<U12')